# Implement the AI Game Strategy

#Part 1 – Install the Python Libraries required for Game Strategy New Section

In [76]:
from collections import namedtuple,Counter,defaultdict
import random
import math
import functools
cache=functools.lru_cache(10**7)  #Least recently used cache

In [77]:
class AI_Game:
    '''A AI_Game is similar to a problem, but it has a terminal test instead of a goal test, and a utility for each terminal state. To create a AI_Game,
       subclass this class and implement `Act`, `result`, `is_terminal`, and `utility`. You will also need to set the .initial attribute to the initial state;
       this can be done in the constructor.'''

    def Act(self,state):
        raise Not_Implemented_Error #Return a collection of the allowable moves from this state

    def result(self,state,move):
        raise Not_Implemented_Error #Return the state that results from making a move from a state

    def is_terminal(self,state):
        return not self.Act(state) #Return True if this is a final state for the AI_Game

    def utility(self,state,player):
        raise Not_Implemented_Error #Return the value of this final state to player


In [78]:
def Lets_Play_AI_Game(AI_Game,strategy:dict,verbose=False):
    '''Play a turn-taking AI_Game. `strategy` is a {player_name: function} dict, where function(state, AI_Game) is used to get the player's move.'''
    state=AI_Game.initial
    while not AI_Game.is_terminal(state):
        player=state.to_move
        move=strategy[player](AI_Game,state)
        state=AI_Game.result(state,move)
        if verbose:
            print('Player',player,'move:',move)
            print(state)
    return state

#Part 2 – Implement the Game Strategy Algorithms


In [79]:
def Minmax_Search(AI_Game,state):
    '''Search AI_Game tree to determine best move; return (value, move) pair.'''

    player=state.to_move

    def Max_value(state):
        if AI_Game.is_terminal(state):
            return AI_Game.utility(state,player),None
        v,move = -infinity,None
        for a in AI_Game.Act(state):
            v2, _ = Min_Value(AI_Game.result(state,a))
            if v2>v:
                v,move=v2,a
        return v,move

    def Min_Value(state):
        if AI_Game.is_terminal(state):
            return AI_Game.utility(state,player),None
        v,move = +infinity,None
        for a in AI_Game.Act(state):
            v2, _ = Max_value(AI_Game.result(state,a))
            if v2<v:
                v,move = v2,a
        return v,move

    return Max_value(state)

infinity = math.inf

In [80]:
def Alpha_Beta_Search(AI_Game,state):
    '''Search AI_Game to determine best action; use alpha-beta pruning.
       Search all the way to the leaves.'''

    player=state.to_move

    def Max_value(state,alpha,beta):
        if AI_Game.is_terminal(state):
            return AI_Game.utility(state,player),None
        v,move = -infinity,None
        for a in AI_Game.Act(state):
            v2, _ = Min_Value(AI_Game.result(state,a),alpha,beta)
            if v2>v:
                v,move = v2,a
                alpha=max(alpha, v)
            if v>=beta:
                return v,move
        return v,move

    def Min_Value(state,alpha,beta):
        if AI_Game.is_terminal(state):
            return AI_Game.utility(state,player),None
        v,move = +infinity,None
        for a in AI_Game.Act(state):
            v2, _ = Max_value(AI_Game.result(state,a),alpha,beta)
            if v2<v:
                v,move = v2,a
                beta=min(beta,v)
            if v<=alpha:
                return v,move
        return v,move

    return Max_value(state, -infinity, +infinity)


# Part 3 – Implement the Game Strategy using TicTocToe

In [81]:
class Tic_Tac_Toe(AI_Game):
    '''Play Tic_Tac_Toe on an `height` by `width` board, needing `k` in a row to win.
       'X' plays first against 'O'. '''

    def __init__(self,height=3,width=3,k=3):
        self.k=k # k in a row
        self.squares={(x,y) for x in range(width) for y in range(height)}
        self.initial=Board(height=height,width=width,to_move='X',utility=0)

    def Act(self,board):
        '''Legal moves are any square not yet taken'''
        return self.squares-set(board)

    def result(self,board,square):
        '''Place a marker for current player on square'''
        player=board.to_move
        board=board.new({square:player},to_move=('O' if player=='X' else 'X'))
        win=k_in_row(board,player,square,self.k)
        board.utility=(0 if not win else +1 if player=='X' else -1)
        return board

    def utility(self,board,player):
        '''Return the value to player; 1 for win, -1 for loss, 0 otherwise'''
        return board.utility if player=='X' else -board.utility

    def is_terminal(self,board):
        '''A board is a terminal state if it is won or there are no empty squares'''
        return board.utility!=0 or len(self.squares)==len(board)

    def display(self,board):print(board)


def k_in_row(board,player,square,k):
    '''True if player has k pieces in a line through square'''
    def in_row(x,y,dx,dy): return 0 if board[x,y]!=player else 1 + in_row(x+dx, y+dy, dx, dy)
    return any(in_row(*square,dx,dy) + in_row(*square,-dx,-dy)-1>=k
               for (dx,dy) in ((0,1),(1,0),(1,1),(1,-1)))


In [82]:
class Board(defaultdict):
    '''A board has the player to move, a cached utility value, and a dict of {(x, y): player} entries, where player is 'X' or 'O'.'''
    empty = '.'
    off = '#'

    def __init__(self,width=8,height=8,to_move=None,**kwds):
        self.__dict__.update(width=width,height=height,to_move=to_move,**kwds)

    def new(self,changes:dict,**kwds) -> 'Board':
        "Given a dict of {(x, y): contents} changes, return a New Board with the changes."
        board=Board(width=self.width,height=self.height,**kwds)
        board.update(self)
        board.update(changes)
        return board

    def __missing__(self,loc):
        x,y=loc
        if 0<=x<self.width and 0<=y<self.height:
            return self.empty
        else:
            return self.off

    def __hash__(self):
        return hash(tuple(sorted(self.items()))) + hash(self.to_move)

    def __repr__(self):
        def row(y): return ' '.join(self[x, y] for x in range(self.width))
        return '\n'.join(map(row,range(self.height))) + '\n'


In [83]:
def Random_Player(AI_Game,state): return random.choice(list(AI_Game.Act(state)))

def player(search_algorithm):
    '''A AI_Game player who uses the specified search algorithm'''
    return lambda AI_Game,state:search_algorithm(AI_Game,state)[1]

#Part 4 – Evaluate the AI Game Strategy using TicTocToe


In [84]:
Lets_Play_AI_Game(Tic_Tac_Toe(),dict(X=Random_Player,O=player(Alpha_Beta_Search)),verbose=True).utility

Player X move: (1, 2)
. . .
. . .
. X .

Player O move: (1, 1)
. . .
. O .
. X .

Player X move: (0, 0)
X . .
. O .
. X .

Player O move: (0, 1)
X . .
O O .
. X .

Player X move: (2, 0)
X . X
O O .
. X .

Player O move: (2, 1)
X . X
O O O
. X .



-1

In [85]:
Lets_Play_AI_Game(Tic_Tac_Toe(),dict(X=player(Alpha_Beta_Search),O=player(Minmax_Search)),verbose=True).utility

Player X move: (0, 1)
. . .
X . .
. . .

Player O move: (2, 1)
. . .
X . O
. . .

Player X move: (1, 2)
. . .
X . O
. X .

Player O move: (0, 0)
O . .
X . O
. X .

Player X move: (1, 1)
O . .
X X O
. X .

Player O move: (1, 0)
O O .
X X O
. X .

Player X move: (2, 0)
O O X
X X O
. X .

Player O move: (0, 2)
O O X
X X O
O X .

Player X move: (2, 2)
O O X
X X O
O X X



0